# 4. Types for agents

We are getting towards our goal of putting everything together to make a model of people moving around and interacting with one another. Most people start off susceptible, but when an infectious person meets a susceptible the infection is transmitted with a certain probability

We will make an **individual-based model**, also called an **agent-based model**. We need a `struct` called `Agent` that contains whatever information an agent needs. In our case we will need a position and an infection status.

The position will behave almost like a normal random walk that we have seen before, while the infection status needs to reflect whether the agent is susceptible (S), infectious (I) or recovered / removed (R).

## Enums

We could represent the infection status simply using an integer, e.g. 0, 1 or 2. 
But then our code will be hard to read, since we will be comparing the infection status to numbers all the time without remembering which one is which.

Rather we would prefer to have the symbols `S`, `I` and `R`. 
We could just define global constants for these, but there is a better alternative: an `Enum`.

`Enum` is short for "*enum*erated type". We want a new *type*, `InfectionStatus`, with possible values `S`, `I` and `R`. 
    We can achieve this as follows:

In [2]:
@enum InfectionStatus S=1 I R   # specify that `S` corresponds to the value 1 

In [3]:
I

I::InfectionStatus = 2

We see that I is an object of type `InfectionStatus`, which has the value 2, in the sense that we can convert it to an integer:

In [5]:
status = I
Int(status)

2

We can check the status with

In [6]:
if status == I
    println("infected!")
end

infected!


In our application the integer values corresponding to each state are arbitrary, but in other applications you may want to give meaning to the numerical values.

## Agent type: Inheritance

How can we create an `Agent` type for an agent that lives in 2D? Perhaps the simplest way is as follows:

In [7]:
struct SimpleAgent1
    x::Int
    y::Int
    status::InfectionStatus
end

[Note that in Julia you cannot redefine a `struct` to have different fields in the same session. We will make several versions so we are just adding a number to each one.]

#### Exercise

1. Write a method of `move` for `SimpleAgent1`.

The problem with this is that we have duplicated codfe: `move` for `SimpleAgent1` looks identical to the one for `Walker2D`. 

One way to reduce duplication is using a form of **inheritance**, by making both types **subtypes** of a common type. The subtype relation is written `X <: Y` (read "`X` is a subtype of `Y`"). The common type is an **abstract type**:

In [8]:
abstract type AbstractWalker2D end

In [14]:
struct SimpleWalker2D <: AbstractWalker2D
    x::Int64
    y::Int64
end

In [15]:
struct SimpleAgent2 <: AbstractWalker2D
    x::Int64
    y::Int64
    status::InfectionStatus
end

Since these two types share common fields, we can write the `move` method just *once*, giving the abstract type as the type annotation.

#### Exercise

1. How can you create an object of type `SimpleAgent2`?


2. Can you extend the `move` function such that it works for both `SimpleWalker2D` and `SimpleAgent2`?

## Composition

An alternative method, which [is usually more powerful and general](https://en.wikipedia.org/wiki/Composition_over_inheritance), is to use **composition**, i.e. placing one object *inside* another:

In [13]:
struct Agent1
    position::SimpleWalker2D
    status::InfectionStatus
end

At first glance this may look a little strange, since we seem to be saying that an agent contains a walker. But really it is just a way to maintain the data inside its pacakge and be able to reuse the infrastructure that we have already built up for that type.

#### Exercise

1. How can you construct an object of type `Agent1`?


2. How would you extend `move` for `Agent1`?

## Parametrised types

However, now suppose we want to have agents that live in 1D or 2D, or on a network etc. Currently we would have to make different types `Agent2D`, `Agent1D`, ... that look almost identical.

We have already seen a few times that whenever we observe **code duplication** like this, it's a hint that there is a possible **abstraction**. 
Basically we want to be able to say "an agent has some kind of position, and an infection status".

We could write this as follows:

In [ ]:
struct Agent2
    position
    status::InfectionStatus
end

with no type annotation on `position`. However, it turns out that this is *very bad for performance* in Julia.

> Never define a composite type with untyped or abstractly-typed fields. 

[Unless you really know what you are doing, or don't care at all about performance.]

To get high performance we need to always allow Julia to be able to work out the types of everything.

The correct solution is to use a **parametrised type**:

In [ ]:
struct Agent3{P}
    position::P
    status::InfectionStatus
end

Here, `P` is a **type parameter**. You can think of it as a special kind of variable that holds a type. Julia will work out what this type should be, based on the kind of object you supply for the `position` field.

*If* there is a common abstract type `AbstractWalker` for all of the possible types that we want to be able to use for `P`, then we can instead write

In [ ]:
struct Agent4{P <: AbstractWalker}
    position::P
    status::InfectionStatus
end

This *restricts* the allowed types that can be used for `P`, which is often desirable. However, it may exclude other possible types, for example `Int`, which cannot be made into a subtype of `AbstractWalker`. 
In this case the previous, unrestricted version may be better.

[You can also use `Union` to specify different allowed types.]

#### Exercise

1. Construct objects of type `Agent3` with different types for `P`. Hint: You *don't need* to explicitly tell Julia the type parameter &ndash; it should be worked out automatically!